In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import re
from scipy.stats import spearmanr
import seaborn as sns
from scipy.spatial import distance

In [ ]:
#for p2 food category
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
directory_path = os.path.join(desktop_path, 'human_project_data')
file_path = os.path.join(directory_path, 'for p2 food category correlation coefficient.xlsx')
df = pd.read_excel(file_path)

col12 = df.iloc[5] == 12
col13 = df.iloc[5] == 13
col14 = df.iloc[5] == 14
col23 = df.iloc[5] == 23
col24 = df.iloc[5] == 24
col34 = df.iloc[5] == 34
colp2only = df.iloc[4] == 'p2-only'
colp2 = df.iloc[4] == 'p2'

data12_p2only = df.loc[0:4,col12 & colp2only]
data13_p2only = df.loc[0:4,col13 & colp2only]
data14_p2only = df.loc[0:4,col14 & colp2only]
data23_p2only = df.loc[0:4,col23 & colp2only]
data24_p2only = df.loc[0:4,col24 & colp2only]
data34_p2only = df.loc[0:4,col34 & colp2only]

data12_p2 = df.loc[0:4,col12 & colp2]
data13_p2 = df.loc[0:4,col13 & colp2]
data14_p2 = df.loc[0:4,col14 & colp2]
data23_p2 = df.loc[0:4,col23 & colp2]
data24_p2 = df.loc[0:4,col24 & colp2]
data34_p2 = df.loc[0:4,col34 & colp2]

order12 = [0,1,2,3]
order13 = [0,2,1,3]
order14 = [0,3,1,2]
order23 = [1,2,0,3]
order24 = [1,3,0,2]
order34 = [2,3,0,1]

d1 = data12_p2only_sorted = data12_p2only.iloc[order12].reset_index(drop=True)
d2 = data13_p2only_sorted = data13_p2only.iloc[order13].reset_index(drop=True)
d3 = data14_p2only_sorted = data14_p2only.iloc[order14].reset_index(drop=True)
d4 = data23_p2only_sorted = data23_p2only.iloc[order23].reset_index(drop=True)
d5 = data24_p2only_sorted = data24_p2only.iloc[order24].reset_index(drop=True)
d6 = data34_p2only_sorted = data34_p2only.iloc[order34].reset_index(drop=True)

d7 = data12_p2_sorted = data12_p2.iloc[order12].reset_index(drop=True)
d8 = data13_p2_sorted = data13_p2.iloc[order13].reset_index(drop=True)
d9 = data14_p2_sorted = data14_p2.iloc[order14].reset_index(drop=True)
d10 = data23_p2_sorted = data23_p2.iloc[order23].reset_index(drop=True)
d11 = data24_p2_sorted = data24_p2.iloc[order24].reset_index(drop=True)
d12 = data34_p2_sorted = data34_p2.iloc[order34].reset_index(drop=True)

data_p2only = pd.concat([d1,d2,d3,d4,d5,d6], axis = 1)
data_p2 = pd.concat([d7,d8,d9,d10,d11,d12], axis = 1)
#保存sort后的被试答案csv
p2only_sort_csv = 'food_dim_p2only_sort.csv'
p2_sort_csv = 'food_dim_p2_sort.csv'
p2only_csv_save_full_name = os.path.join(directory_path, 'after_sorting_csv',p2only_sort_csv)
p2_csv_save_full_name = os.path.join(directory_path, 'after_sorting_csv', p2_sort_csv )
data_p2only.to_csv(p2only_csv_save_full_name)
data_p2.to_csv(p2_csv_save_full_name)

score_dic = df.loc[7].to_dict()

if 'fullanswer12' in df.columns:
    fullanswer = df['fullanswer12'].iloc[0:4]
    print("前4个 'fullanswer12' 的值:", fullanswer)
else:
    print("'fullanswer12' 列不存在")


results_p2only = []
results_p2 = []


file_save_dir = os.path.join(directory_path, 'p2 food category spearman corr')
if not os.path.exists(file_save_dir):
    os.makedirs(file_save_dir) 

# fig, axes = plt.subplots(nrows=data_p2only.shape[1], ncols=1, figsize=(5, 5 * data_p2only.shape[1])) 

for i, column in enumerate(data_p2only.columns):

    # 对 df1 和 df2 中的列进行秩变换，处理重复值
    # ranked_df1 = df1[column].rank(method='average')  # 平均法对重复值赋秩
    # ranked_df2 = df2.iloc[:, 0].rank(method='average')
    data_p2only[column] = pd.to_numeric(data_p2only[column])
    fullanswer = pd.to_numeric(fullanswer)

    corr, _ = spearmanr(data_p2only[column], fullanswer)#计算斯皮尔曼相关系数
    
    #计算欧式距离
    array_sub = data_p2only[column].values
    array_answer = fullanswer.values
    distance_p2only = distance.euclidean(array_sub, array_answer)
    #遍历分数字典找到对应的分数
    if column in score_dic:
        score = score_dic[column]
        print(f"{column} score found!")
    else:
        print(f"{column} no match score!")

    results_p2only.append([column, corr, distance_p2only, score])
    

    # ranked_column1 = np.argsort(np.argsort(data_p2only[column]))
    # ranked_column2 = np.argsort(np.argsort(fullanswer.iloc[:,]))
    
    fig, ax = plt.subplots(figsize=(5,5))

    sns.scatterplot(x=data_p2only[column], y=fullanswer, ax=ax) #ax=axes[i])
    sns.regplot(x=data_p2only[column], y=fullanswer, ax=ax, scatter=False, line_kws={'color': 'orange'})

    ax.set_title(f'Scatter plot with {column} (Spearman corr: {corr:.2f})', fontsize = 13)
    ax.set_xlabel(" Subject's confidence of each food category ")
    ax.set_ylabel('weights of each food category in fullanswer')
    ax.text(0.01,0.99, f'highest score: {score}',fontsize = 15, color = 'red', ha='left', va='top',transform=ax.transAxes )
    
    file_save_name = f'p2-only_spearman corr of {column} for dim identification.png'
    file_save_full_name = os.path.join(file_save_dir, file_save_name)
    plt.savefig(file_save_full_name)
    plt.close




results_p2only_df = pd.DataFrame(results_p2only, columns=['Subject','Correlation coefficient','Distance','Score'])
print(results_p2only_df)

for i, column in enumerate(data_p2.columns):

    # 对 df1 和 df2 中的列进行秩变换，处理重复值
    # ranked_df1 = df1[column].rank(method='average')  # 平均法对重复值赋秩
    # ranked_df2 = df2.iloc[:, 0].rank(method='average')
    data_p2[column] = pd.to_numeric(data_p2[column])
    corr, _ = spearmanr(data_p2[column], fullanswer)

    array_sub = data_p2[column].values
    array_answer = fullanswer.values
    distance_p2 = distance.euclidean(array_sub, array_answer)

    #遍历分数字典找到对应的分数
    if column in score_dic:
        score = score_dic[column]
        print(f"{column} score found!")
    else:
        print(f"{column} no match score!")

    results_p2only.append([column, corr, distance_p2only, score])

    results_p2.append([column, corr, distance_p2, score])

    fig, ax = plt.subplots(figsize=(5,5))

    sns.scatterplot(x=data_p2[column], y=fullanswer, ax=ax) #ax=axes[i])
    sns.regplot(x=data_p2[column], y=fullanswer, ax=ax, scatter=False, line_kws={'color': 'orange'})

    ax.set_title(f'Scatter plot with {column} (Spearman corr: {corr:.2f})')
    ax.set_xlabel(" Subject's confidence of each food category ")
    ax.set_ylabel('weights of each food category in fullanswer')
    ax.text(0.01,0.99, f'highest score: {score}',fontsize = 15, color = 'red', ha='left', va='top',transform=ax.transAxes )
    file_save_name = f'p2_spearman corr of {column} for dim identification.png'
    file_save_full_name = os.path.join(file_save_dir, file_save_name)
    plt.savefig(file_save_full_name)
    plt.close
    




results_p2_df = pd.DataFrame(results_p2, columns=['Subject','Correlation coefficient','Distance','Score'])
print(results_p2_df)

#保存csv文件
p2only_csv_save_name = 'result_p2only_dim.csv'
p2_csv_save_name = 'result_p2_dim.csv'
p2only_csv_save_full_name = os.path.join(file_save_dir, p2only_csv_save_name)
p2_csv_save_full_name = os.path.join(file_save_dir, p2_csv_save_name)
results_p2only_df.to_csv(p2only_csv_save_full_name)
results_p2_df.to_csv(p2_csv_save_full_name)


nd1 = p2_only_cc = results_p2only_df['Correlation coefficient'].to_numpy()
nd2 = p2_cc = results_p2_df['Correlation coefficient'].to_numpy()
nd1_clean = nd1[~np.isnan(nd1)]
nd2_clean = nd2[~np.isnan(nd2)]
mean_nd1 = np.mean(nd1_clean)
mean_nd2 = np.mean(nd2_clean)

# bin_width = 0.1
# min_value = min(np.min(nd1), np.min(nd2))  # 获取数据的最小值
# max_value = max(np.max(nd1), np.max(nd2))  # 获取数据的最大值

# 创建更小区间的bins，增大柱子的数量
# bins = np.arange(min_value, max_value + bin_width, bin_width)

fig, ax = plt.subplots()

# Plot the first histogram
ax.hist(nd1, label="p2-only group")

# Plot the second histogram
# (notice the negative weights, which flip the histogram upside down)
ax.hist(nd2, weights=-np.ones_like(nd2),  label="p1p2 group - p2")
ax.set_xlabel(" Spearman correlation coefficient ")
ax.set_ylabel('Counts of subjects')
ax.axvline(mean_nd1, color='blue', linestyle='dashed', linewidth=2, label=f'Mean p2-only: {mean_nd1:.2f}')
ax.axvline(mean_nd2, color='red', linestyle='dashed', linewidth=2, label=f'Mean p1p2 - p2: {mean_nd2:.2f}')

ax.axhline(0, color="k")
ax.legend()
ax.set_title('mean spearman corr for p2')

fig_save_name = 'Spearman corr mean and distribution for phase2 food category.png'
fig_save_full_name = os.path.join(file_save_dir, fig_save_name)
plt.savefig(fig_save_full_name)